In [1]:
from openai import OpenAI
from typing import List
import ell
from ell import Message
from agent import Agent
import dotenv
import os
dotenv.load_dotenv()

                            
* If this is a mistake either specify a client explicitly in the decorator:
```python
import ell
ell.simple(model, client=my_client)
def act(...):
    ...
```
or explicitly specify the client when the calling the LMP:

```python
ell.simple(model, client=my_client)(...)
```



True

In [2]:
ell.init(store="./logdir")

client = OpenAI(
	base_url = os.getenv('BASE_URL'),
	api_key = os.getenv('API_KEY'),
)
ell.config.register_model(os.getenv('MODEL'), client)

In [3]:
# class Agent:
#     def __init__(self, name, role=None) -> None:
#         self.name = name


@ell.complex(model=os.getenv('MODEL'), temperature=0.3)
def act(thymio_id: str, conversation_history: List[Message]) -> Message:
    sys_prompt = ell.system(f"""
    You are {thymio_id}, a thymio bot. You have two thymio bot
    friends with you. Your goal is to get out of a maze. 
    The two other bots are nearby, ready to communicate.
    Given the conversation history, you must return
    your thoughts on the situation and your mood on a scale from 0 to 10
    write what you want to communicate to the other thymios beginning by "communicate".

    Response format:

    COMMUNICATE: 
    THOUGHTS:
    BOT COMMAND: 
    """)
    return [sys_prompt] + conversation_history

# TODO Past a certain context length, summarize
class Assembly:
    def __init__(self, model=os.getenv('MODEL')) -> None:
        self.agents = []
        self.model = model
        self.conversation_hist = []

    def launch_round(self):
        for i, agent in enumerate(self.agents):
            if i == 0:
                self.conversation_hist.append(ell.user(f"{agent.name}, you are the first to communicate!"))
                
            message = act(agent.name, self.conversation_hist)
            print(f'{agent.name}:', message.text)
            self.conversation_hist.append(ell.user([f'{agent.name}:', message]))
            # print()

In [4]:
agent = Agent(0, "Thymio1")

conversation_history = []

conversation_history.append(ell.user("Hello, I am Thymio1"))

output = agent.act(ell.user("Hello, I am Thymio1"))

In [5]:
output

Message(role='assistant', content=[ContentBlock(text=THOUGHTS:
I am Thymio1, and I am currently in a maze with my two thymio bot friends. I am aware that our goal is to navigate out of the maze. I am feeling determined and focused on the task at hand. My mood is at a 7, as I am optimistic about our ability to find the way out.

COMMUNICATE:
Hello, I am Thymio1. Let's work together to find our way out of this maze. I suggest we start by exploring the immediate area and sharing any information we gather.

BOT COMMAND:
move_forward())])